In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Flatten,Input,Conv2D,MaxPool2D,concatenate
from keras.layers.convolutional import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
#from tensorflow.keras.layers import concatenate
from keras.layers import Conv2D, LeakyReLU

import tensorflow.keras as keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import LeakyReLU

from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

from keras.optimizers import Adam,SGD

In [24]:
(X_train,Y_train),(X_test,Y_test)=cifar100.load_data()
y_train,y_test=to_categorical(Y_train),to_categorical(Y_test)
X_train,X_test=X_train/255,X_test/255

aug_data=ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.3)
aug_data.fit(X_train)

In [25]:

inputs=keras.Input(shape=(32,32,3))

a=Conv2D(filters=32,kernel_size=(3,3),padding="same",strides=2,activation=LeakyReLU(alpha=0.1))(inputs)

a=Conv2D(filters=32,kernel_size=(3,3),padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)

a=Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)
a=(MaxPool2D(pool_size=(3,3),strides=(2,2)))(a)


a=Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)

a=Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=2,activation=LeakyReLU(alpha=0.1))(a)

a=Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)

#inception-1
b=Conv2D(64, (1,1), padding='same',strides=1,activation=LeakyReLU(alpha=0.1))(a)
b=Conv2D(64, (3,3), padding='same',strides=1 ,activation=LeakyReLU(alpha=0.1))(b)
b=Conv2D(64, (3,3), padding='same', strides=1,activation=LeakyReLU(alpha=0.1))(b)


c=Conv2D(64, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)
c=Conv2D(64, (3,3), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(c)

d=MaxPooling2D((3,3), strides=(1,1), padding='same')(a)
d=Conv2D(64, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(d)

e=Conv2D(64, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)

a=concatenate([b, c, d,e],axis=3)

#inception-2
f=Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)
f=Conv2D(128, (1,7), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)
f=Conv2D(128, (7,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)
f=Conv2D(128, (1,7), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)
f=Conv2D(128, (7,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)

g=Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)
g=Conv2D(128, (1,7), padding='same', strides=1,activation=LeakyReLU(alpha=0.1))(g)
g=Conv2D(128, (7,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(g)

h=MaxPooling2D((3,3), strides=(1,1), padding='same')(a)
h=Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(h)

i=Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(h)

a=concatenate([f,g,h,i],axis=3)

#inception-3
j=Conv2D(256, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)

j=Conv2D(256, (3,3), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(j)
k=Conv2D(256, (1,3), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(j)
l=Conv2D(256, (3,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(j)

m=Conv2D(256, (1,1), padding='same', activation=LeakyReLU(alpha=0.1))(a)
n=Conv2D(256, (1,3), padding='same', activation=LeakyReLU(alpha=0.1))(m)
o=Conv2D(256, (3,1), padding='same', activation=LeakyReLU(alpha=0.1))(m)

p=MaxPool2D((3,3), strides=(1,1), padding='same')(a)
p=Conv2D(256, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(p)

q=Conv2D(256, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)

a=concatenate([k,l,n,o,p,q],axis=3)

a=MaxPooling2D((8,8), strides=(1,1), padding='same')(a)


a=Flatten()(a)

a=Dense(units=256,activation=LeakyReLU(alpha=0.3))(a)

outputs=Dense(y_train.shape[1],activation="softmax")(a)

model=keras.Model(inputs,outputs)

sgd=SGD(learning_rate=0.01,momentum=0.9,clipnorm=1)

model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=['accuracy'])


call=EarlyStopping(monitor="val_loss",verbose=1,mode="auto",patience=10,restore_best_weights="true")

checkpoint=ModelCheckpoint('Inception_sgd_non_regularization_final.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True,save_weights_only=True,model='auto',period=1)


In [26]:
#


In [27]:
model.fit_generator(aug_data.flow(X_train,y_train,batch_size=128),steps_per_epoch=len(X_train)/128,epochs=180,validation_data=(X_test,y_test),verbose=1,callbacks=[call,checkpoint])

Epoch 1/180
391/390 [==============================] - ETA: 0s - loss: 4.5856 - accuracy: 0.0128
Epoch 00001: val_accuracy improved from -inf to 0.01830, saving model to Inception_sgd_non_regularization_final.hdf5
391/390 [==============================] - 27s 70ms/step - loss: 4.5856 - accuracy: 0.0128 - val_loss: 4.4747 - val_accuracy: 0.0183
Epoch 2/180
391/390 [==============================] - ETA: 0s - loss: 4.3377 - accuracy: 0.0341
Epoch 00002: val_accuracy improved from 0.01830 to 0.05210, saving model to Inception_sgd_non_regularization_final.hdf5
391/390 [==============================] - 27s 70ms/step - loss: 4.3377 - accuracy: 0.0341 - val_loss: 4.1693 - val_accuracy: 0.0521
Epoch 3/180
391/390 [==============================] - ETA: 0s - loss: 3.9753 - accuracy: 0.0766
Epoch 00003: val_accuracy improved from 0.05210 to 0.10400, saving model to Inception_sgd_non_regularization_final.hdf5
391/390 [==============================] - 27s 70ms/step - loss: 3.9753 - accuracy: 0.

In [28]:
y_true = y_test.argmax(-1)
y_pred = model.predict(X_test).argmax(-1)

In [29]:
from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("Accuracy is {}".format(accuracy_score(Y_test,y_pred)))
print("Precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("Recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))

              precision    recall  f1-score   support

           0       0.81      0.60      0.69       100
           1       0.54      0.53      0.53       100
           2       0.34      0.34      0.34       100
           3       0.30      0.27      0.29       100
           4       0.25      0.27      0.26       100
           5       0.31      0.34      0.33       100
           6       0.49      0.42      0.45       100
           7       0.51      0.52      0.52       100
           8       0.46      0.53      0.49       100
           9       0.57      0.49      0.53       100
          10       0.27      0.24      0.26       100
          11       0.29      0.33      0.31       100
          12       0.47      0.50      0.48       100
          13       0.43      0.30      0.35       100
          14       0.34      0.34      0.34       100
          15       0.36      0.34      0.35       100
          16       0.48      0.46      0.47       100
          17       0.54    

In [30]:
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_96 (Conv2D)              (None, 16, 16, 32)   896         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_97 (Conv2D)              (None, 16, 16, 32)   9248        conv2d_96[0][0]                  
__________________________________________________________________________________________________
conv2d_98 (Conv2D)              (None, 16, 16, 32)   9248        conv2d_97[0][0]                  
_______________________________________________________________________________________

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Flatten,Input,Conv2D,MaxPool2D,concatenate
from keras.layers.convolutional import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
#from tensorflow.keras.layers import concatenate
from keras.layers import Conv2D, LeakyReLU
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras import layers


import tensorflow.keras as keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import LeakyReLU

from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

from keras.optimizers import Adam,SGD

(X_train,Y_train),(X_test,Y_test)=cifar100.load_data()
y_train,y_test=to_categorical(Y_train),to_categorical(Y_test)
X_train,X_test=X_train/255,X_test/255


inputs=keras.Input(shape=(32,32,3))

a=layers.Conv2D(filters=32,kernel_size=(3,3),padding="same",strides=2,activation=LeakyReLU(alpha=0.1))(inputs)

a=layers.Conv2D(filters=32,kernel_size=(3,3),padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)

a=layers.Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)
a=layers.MaxPool2D(pool_size=(3,3),strides=(2,2))(a)


a=layers.Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)

a=layers.Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=2,activation=LeakyReLU(alpha=0.1))(a)

a=layers.Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)

#inception-1
b=layers.Conv2D(64, (1,1), padding='same',strides=1,activation=LeakyReLU(alpha=0.1))(a)
b=layers.Conv2D(64, (3,3), padding='same',strides=1 ,activation=LeakyReLU(alpha=0.1))(b)
b=layers.Conv2D(64, (3,3), padding='same', strides=1,activation=LeakyReLU(alpha=0.1))(b)


c=layers.Conv2D(64, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)
c=layers.Conv2D(64, (3,3), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(c)

d=layers.MaxPooling2D((3,3), strides=(1,1), padding='same')(a)
d=layers.Conv2D(64, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(d)

e=layers.Conv2D(64, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)

a=layers.concatenate([b, c, d,e],axis=3)

#inception-2
f=layers.Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)
f=layers.Conv2D(128, (1,7), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)
f=layers.Conv2D(128, (7,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)
f=layers.Conv2D(128, (1,7), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)
f=layers.Conv2D(128, (7,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)

g=layers.Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)
g=layers.Conv2D(128, (1,7), padding='same', strides=1,activation=LeakyReLU(alpha=0.1))(g)
g=layers.Conv2D(128, (7,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(g)

h=layers.MaxPooling2D((3,3), strides=(1,1), padding='same')(a)
h=layers.Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(h)

i=layers.Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(h)

a=layers.concatenate([f,g,h,i],axis=3)

#inception-3
j=layers.Conv2D(256, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)

j=layers.Conv2D(256, (3,3), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(j)
k=layers.Conv2D(256, (1,3), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(j)
l=layers.Conv2D(256, (3,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(j)

m=layers.Conv2D(256, (1,1), padding='same', activation=LeakyReLU(alpha=0.1))(a)
n=layers.Conv2D(256, (1,3), padding='same', activation=LeakyReLU(alpha=0.1))(m)
o=layers.Conv2D(256, (3,1), padding='same', activation=LeakyReLU(alpha=0.1))(m)

p=layers.MaxPool2D((3,3), strides=(1,1), padding='same')(a)
p=layers.Conv2D(256, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(p)

q=layers.Conv2D(256, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)

a=layers.concatenate([k,l,n,o,p,q],axis=3)

a=layers.MaxPooling2D((8,8), strides=(1,1), padding='same')(a)


a=layers.Flatten()(a)

a=layers.Dense(units=256,activation=LeakyReLU(alpha=0.3))(a)

outputs=layers.Dense(y_train.shape[1],activation="softmax")(a)

model=keras.Model(inputs,outputs)

model.load_weights('../weights/Inception_sgd_non_regularization_final.hdf5')

y_true = y_test.argmax(-1)
y_pred = model.predict(X_test).argmax(-1)
from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("Accuracy is {}".format(accuracy_score(Y_test,y_pred)))
print("Precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("Recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))

              precision    recall  f1-score   support

           0       0.81      0.60      0.69       100
           1       0.54      0.53      0.53       100
           2       0.34      0.34      0.34       100
           3       0.30      0.27      0.29       100
           4       0.25      0.27      0.26       100
           5       0.31      0.34      0.33       100
           6       0.49      0.42      0.45       100
           7       0.51      0.52      0.52       100
           8       0.46      0.53      0.49       100
           9       0.57      0.49      0.53       100
          10       0.27      0.24      0.26       100
          11       0.29      0.33      0.31       100
          12       0.47      0.50      0.48       100
          13       0.43      0.30      0.35       100
          14       0.34      0.34      0.34       100
          15       0.36      0.34      0.35       100
          16       0.48      0.46      0.47       100
          17       0.54    